- Beeradvocate.com의 TOP 250 읽어오기

In [1]:
#임포트
import requests
from bs4 import BeautifulSoup as bs
import random
import time
from tqdm import tqdm, trange  # 진행 상황
import os
from urllib.request import urlretrieve, Request, urlopen

In [10]:
#링크 받아오기
def get_beer_list():
    base_url = "https://www.beeradvocate.com/beer/popular/"
    response = requests.get (base_url )
    
    if response.status_code == 200:
        soup = bs(response.text, 'lxml')
        tds = soup.find('table').findAll('td', {'class':'hr_bottom_light'})[:-3]
        if not tds:
            return beer_list
        
        else:            
            for td in tds:                
                link = td.find('a')
                if link:                    
                    beer_list.append(link['href'])                
                else:
                    pass
                                       
    return beer_list

In [11]:
#크롤링 실행
beer_list = []
get_beer_list()

['/beer/profile/1199/11757/',
 '/beer/profile/64/2093/',
 '/beer/profile/287/1093/',
 '/beer/profile/863/7971/',
 '/beer/profile/46317/16814/',
 '/beer/profile/199/29619/',
 '/beer/profile/1199/19960/',
 '/beer/profile/287/17112/',
 '/beer/profile/1146/10672/',
 '/beer/profile/112/412/',
 '/beer/profile/64/6108/',
 '/beer/profile/140/276/',
 '/beer/profile/147/88/',
 '/beer/profile/140/30420/',
 '/beer/profile/147/84596/',
 '/beer/profile/43153/92/',
 '/beer/profile/26/64228/',
 '/beer/profile/140/1904/',
 '/beer/profile/22/34/',
 '/beer/profile/220/74986/',
 '/beer/profile/2681/34483/',
 '/beer/profile/259/1708/',
 '/beer/profile/207/645/',
 '/beer/profile/35/104/',
 '/beer/profile/220/916/',
 '/beer/profile/220/49789/',
 '/beer/profile/192/607/',
 '/beer/profile/252/731/',
 '/beer/profile/17981/46363/',
 '/beer/profile/694/15881/',
 '/beer/profile/1199/5441/',
 '/beer/profile/1199/35036/',
 '/beer/profile/64/9086/',
 '/beer/profile/209/754/',
 '/beer/profile/222/695/',
 '/beer/profil

In [12]:
#개수 확인
len(beer_list)

250

In [16]:
#페이지 안에서 크롤링
def get_beer_content(url):
    global data    
    base_url = f"http://beeradvocate.com{url}"
    response = requests.get ( base_url )

    soup = bs(response.text, 'lxml')   

    note = soup.select("#ba-content > div:nth-child(8) > div:nth-child(3)")[0].text
    note2 = note.split(":")[-1].strip() 

    titles = soup.find("div", {'class':'titleBar'}).contents[1]    
    title_span = titles.find('span').text    
    title = titles.text.replace(title_span,"")    

    stats = soup.findAll("dd", {'class':'beerstats'})        

    beerData = {"beerNo":url.split("/")[-2], "beerName":title, "type":(stats[0].find('b')).text,
                "country":stats[7].text.split(', ')[-1], "company":stats[6].text,
            "ABV":stats[1].text.split('%')[0], "ratingBA":stats[3].text.split(' ')[0],
            "ratingCnt":stats[5].text.replace(',',''), "Notes":note2}

    return beerData

In [17]:
#크롤링 실행
beer_datas = []
for i, beers in tqdm(enumerate(beer_list)):
    b = get_beer_content(beers)
    beer_datas.append(b)
    time.sleep(random.randint(1, 2))

250it [13:50,  3.32s/it]


In [18]:
#확인
len(beer_datas)

250

In [37]:
beer_datas[102]

{'beerNo': '449',
 'beerName': 'Stella Artois',
 'type': 'European Pale Lager',
 'country': 'Belgium',
 'company': 'Stella Artois',
 'ABV': ' 5',
 'ratingBA': '3.11',
 'ratingCnt': '5749',
 'Notes': ''}

In [38]:
#오라클 연결
import cx_Oracle

connstr = 'bgirls/BGirls@192.168.1.46:1521/orcl'
conn = cx_Oracle.connect(connstr)
curs = conn.cursor()
os.environ["NLS_LANG"] = ".AL32UTF8"

In [39]:
#저장하기
for bd in beer_datas:  
        
    sql = 'INSERT INTO beer(beerNo, beerName, country, company, type, abv, rating, ratingBA, ratingCnt, note) VALUES(:beerNo, :beerName, :country, :company, :type, :abv, :rating, :ratingBA, :ratingCnt, :note)'    
    
    try:
        curs.prepare(sql)
        curs.execute(None, beerNo=bd['beerNo'], beerName=bd['beerName'], country=bd['country'], company=bd['company'], type=bd['type'], abv=float(bd['ABV']), rating=0, ratingBA=float(bd['ratingBA']), ratingCnt=int(bd['ratingCnt']), note=bd['Notes'])        
        conn.commit() 
        
    except Exception as e:
        if str(e).split(':')[0] != 'ORA-00001':
            
            print(e, '::', bd['beerName'], bd['beerNo'], bd['company'])
            conn.rollback()
        else:
            conn.rollback()
        
        
curs.close()        
conn.close()


In [40]:
# cdn 사진 별도 저장
#사진 저장 메소드
def get_beer_pic(url):
    global data    
    base_url = f"http://beeradvocate.com{url}"
    response = requests.get ( base_url )
    
    if response.status_code == 200:     
        
        soup = bs(response.text, 'lxml')
        pic = soup.findAll('img')[2].get('src')        
        fname = pic.split("/")[-1]
        
        #f = open(fname,'wb')
        #f.write(urllib.request.urlopen(pic).read())
        #f.close()        
        
        f = open(fname, 'wb')
        req = Request(pic, headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        f.write(webpage)
        f.close() 
                
        #urlretrieve(pic, fname)
        print("이미지 저장 완료")       
        

In [41]:
# 사진 저장 크롤링
for i, beer in tqdm(enumerate(beer_list)):
    b = get_beer_pic(beer)    
    time.sleep(random.randint(1, 2))

0it [00:00, ?it/s]

이미지 저장 완료


1it [00:03,  3.54s/it]

이미지 저장 완료


2it [00:07,  3.73s/it]

...

248it [16:45,  3.71s/it]

이미지 저장 완료


249it [16:49,  3.71s/it]

이미지 저장 완료


250it [16:52,  4.05s/it]
